## Practicing Joining Data with pandas

### Practicing on August 27th, 2025

### Loading Packages

In [1]:
### importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Creating a customer table

In [2]:

customers = pd.DataFrame({
    "customer_id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Charlie", "Diana"],
    "country": ["USA", "UK", "USA", "Canada"]
})
print(customers)


   customer_id     name country
0            1    Alice     USA
1            2      Bob      UK
2            3  Charlie     USA
3            4    Diana  Canada


### Creating an order table

In [3]:
orders = pd.DataFrame({
    "order_id": [101, 102, 103, 104, 105],
    "customer_id": [1, 2, 2, 5, 3],
    "amount": [250, 150, 300, 200, 450]
})
print(orders)


   order_id  customer_id  amount
0       101            1     250
1       102            2     150
2       103            2     300
3       104            5     200
4       105            3     450


### Inner Join

In [4]:
customer_order = pd.merge(customers, orders, on = "customer_id", how = "inner", suffixes = ("_c", "_o"))

Besides these parameters from above, there is another one validate = None (default) for the pd.merge() or DataFrame.merge()
The validate parameter could test "one_to_one", "one_to_many", "many_to_one", "many_to_many"
if the execution of the merge and validate the results of two table does not match with the validation specified in the parameter then, it would raise merge error.

In [5]:
customer_order

,customer_id,name,country,order_id,amount
0,1,Alice,USA,101,250
1,2,Bob,UK,102,150
2,2,Bob,UK,103,300
3,3,Charlie,USA,105,450


In [6]:
### Find which customers have placed orders, and show their names with order details.
customer_order = pd.merge(customers, orders, on = "customer_id", how = "inner", suffixes = ("_c", "_o"))
customer_order.sort_values(by = "name", ascending = True)

,customer_id,name,country,order_id,amount
0,1,Alice,USA,101,250
1,2,Bob,UK,102,150
2,2,Bob,UK,103,300
3,3,Charlie,USA,105,450


In [11]:
### Find all customers (even those who never ordered) and include their orders if available.
customers_all_order = pd.merge(customers, orders, on = "customer_id", how = "left", suffixes = ("_c", "_o"))
customers_all_order

### modification to how = "left" is good enough

,customer_id,name,country,order_id,amount
0,1,Alice,USA,101.0,250.0
1,2,Bob,UK,102.0,150.0
2,2,Bob,UK,103.0,300.0
3,3,Charlie,USA,105.0,450.0
4,4,Diana,Canada,NaN,NaN


In [10]:
### Find all orders (including ones from customers not in the customers table).
customers_all_order_2 = pd.merge(customers, orders, on = "customer_id", how = "outer", suffixes = ("_c", "_o"))
customers_all_order_2

,customer_id,name,country,order_id,amount
0,1,Alice,USA,101.0,250.0
1,2,Bob,UK,102.0,150.0
2,2,Bob,UK,103.0,300.0
3,3,Charlie,USA,105.0,450.0
4,4,Diana,Canada,NaN,NaN
5,5,NaN,NaN,104.0,200.0


First version: groupby("country").agg('sum')[["amount"]]

Groups by country
Aggregates ALL numeric columns with sum
Then selects only the amount column from the result


Second version: groupby("country")[["amount"]].agg('sum')

Groups by country
Selects the amount column first
Then aggregates only that column with sum

In [13]:
### For each country, calculate the total spending from their customers’ orders.
customers_all_order_2.groupby("country").agg('sum')[["amount"]]

,amount
country,
Canada,0.0
UK,450.0
USA,700.0


In [14]:

customers_all_order_2.groupby("country")["amount"].sum()

country
Canada      0.0
UK        450.0
USA       700.0
Name: amount, dtype: float64

### Loading Movies and Movie Taglines

In [18]:
import pandas as pd

tmdb_movie = pd.read_pickle("/Users/auroraxiao/Desktop/Data Science Project/DS-Projects/Data Set/movies.p")
print(tmdb_movie.head())


      id                 title  popularity release_date
0    257          Oliver Twist   20.415572   2005-09-23
1  14290  Better Luck Tomorrow    3.877036   2002-01-12
2  38365             Grown Ups   38.864027   2010-06-24
3   9672              Infamous    3.680896   2006-11-16
4  12819       Alpha and Omega   12.300789   2010-09-17


In [19]:
tmdb_taglines = pd.read_pickle("/Users/auroraxiao/Desktop/Data Science Project/DS-Projects/Data Set/taglines.p")
print(tmdb_taglines.head())

       id                                         tagline
0   19995                     Enter the World of Pandora.
1     285  At the end of the world, the adventure begins.
2  206647                           A Plan No One Escapes
3   49026                                 The Legend Ends
4   49529            Lost in our world, found in another.


### left join movies and taglines

In [21]:
movies_taglines = tmdb_movie.merge(tmdb_taglines, how = "left", on = "id", suffixes = ("_movies", "_taglines"))
movies_taglines.head()

,id,title,popularity,release_date,tagline
0,257,Oliver Twist,20.415572,2005-09-23,NaN
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,Never underestimate an overachiever.
2,38365,Grown Ups,38.864027,2010-06-24,Boys will be boys. . . some longer than others.
3,9672,Infamous,3.680896,2006-11-16,There's more to the story than you know
4,12819,Alpha and Omega,12.300789,2010-09-17,A Pawsome 3D Adventure


In [22]:
movies_taglines.sort_values(by = "popularity", ascending= False)

,id,title,popularity,release_date,tagline
4546,211672,Minions,875.581305,2015-06-17,"Before Gru, they had a history of bad bosses"
4343,157336,Interstellar,724.247784,2014-11-05,Mankind was born on Earth. It was never meant ...
1966,293660,Deadpool,514.569956,2016-02-09,Witness the beginning of a happy ending
2423,118340,Guardians of the Galaxy,481.098624,2014-07-30,All heroes start somewhere.
4220,76341,Mad Max: Fury Road,434.278564,2015-05-13,What a Lovely Day.
...,...,...,...,...,...
969,426067,Midnight Cabaret,0.001389,1990-01-01,The hot spot where Satan's waitin'.
3558,325140,Hum To Mohabbat Karega,0.001186,2000-05-26,NaN
1692,65448,Penitentiary,0.001117,1979-12-01,"There's only one way out, and 100 fools stand ..."
828,77156,Alien Zone,0.000372,1978-11-22,Don't you dare go in there!


### Merging with Index

In [25]:
### set index of two tables tmdb_taglines and tmdb_movie to "id" before merging
tmdb_movie.head()
tmdb_movie.set_index("id", inplace = True)

In [27]:
movies_taglines.head()
movies_taglines.set_index("id", inplace = True)

In [28]:
movies_taglines.head()

,title,popularity,release_date,tagline
id,,,,
257,Oliver Twist,20.415572,2005-09-23,NaN
14290,Better Luck Tomorrow,3.877036,2002-01-12,Never underestimate an overachiever.
38365,Grown Ups,38.864027,2010-06-24,Boys will be boys. . . some longer than others.
9672,Infamous,3.680896,2006-11-16,There's more to the story than you know
12819,Alpha and Omega,12.300789,2010-09-17,A Pawsome 3D Adventure


In [ ]:


tmdb_taglines.set_index("id", inplace = True)

In [30]:
### merging with index
movies_taglines_2 = tmdb_movie.merge(tmdb_taglines, how = "left", on = "id", suffixes = ("_movies", "_taglines"))

In [31]:
movies_taglines_2.head()

,title,popularity,release_date,tagline
id,,,,
257,Oliver Twist,20.415572,2005-09-23,NaN
14290,Better Luck Tomorrow,3.877036,2002-01-12,Never underestimate an overachiever.
38365,Grown Ups,38.864027,2010-06-24,Boys will be boys. . . some longer than others.
9672,Infamous,3.680896,2006-11-16,There's more to the story than you know
12819,Alpha and Omega,12.300789,2010-09-17,A Pawsome 3D Adventure


### Difference between semi join and inner join

*inner join* :includes columns from both tables, and rows can be duplicated if one side has multiple matches (many-to-one or many-to-many)

*semi-join*: includes only the left table's columns and no duplication from multiple matches, essentially acts like a filter (where exists)

*Anti-join*:  return the left table as well, excluding the interaction (shared rows from both tables)

## Stacking Tables with pd.concat()

In [32]:
# Table 1: Sales data for January
data1 = {
    "CustomerID": [101, 102, 103],
    "Name": ["Alice", "Bob", "Charlie"],
    "Purchase": ["Laptop", "Headphones", "Keyboard"],
    "Amount": [1200, 200, 100]
}

df1 = pd.DataFrame(data1)


# Table 2: Sales data for February
data2 = {
    "CustomerID": [104, 105, 106],
    "Name": ["David", "Eva", "Frank"],
    "Purchase": ["Mouse", "Monitor", "Printer"],
    "Amount": [50, 300, 400]
}

df2 = pd.DataFrame(data2)

print("Table 1:")
print(df1, "\n")

print("Table 2:")
print(df2)

Table 1:
   CustomerID     Name    Purchase  Amount
0         101    Alice      Laptop    1200
1         102      Bob  Headphones     200
2         103  Charlie    Keyboard     100 

Table 2:
   CustomerID   Name Purchase  Amount
0         104  David    Mouse      50
1         105    Eva  Monitor     300
2         106  Frank  Printer     400


In [33]:
## objs: a list or dict of pandas objects (Dataframes or series)
new_combined_table = pd.concat([df1, df2], axis = 0)

In [34]:
new_combined_table.head()

,CustomerID,Name,Purchase,Amount
0,101,Alice,Laptop,1200
1,102,Bob,Headphones,200
2,103,Charlie,Keyboard,100
0,104,David,Mouse,50
1,105,Eva,Monitor,300


There are few other types of merge such as

pd.merge_ordered()

pd.merge_asof()


### DataFrame.query('Statement')

In [35]:
new_combined_table.query('Amount <=1000')

,CustomerID,Name,Purchase,Amount
1,102,Bob,Headphones,200
2,103,Charlie,Keyboard,100
0,104,David,Mouse,50
1,105,Eva,Monitor,300
2,106,Frank,Printer,400


## Unpivot table with DataFrame.melt()

In [37]:

# Create sample wide-format dataframe
data = {
    "Student": ["Alice", "Bob", "Charlie"],
    "Math": [85, 92, 78],
    "Science": [90, 88, 82],
    "History": [70, 95, 80]
}

df_pivot = pd.DataFrame(data)
print("Original DataFrame:\n", df_pivot)


Original DataFrame:
    Student  Math  Science  History
0    Alice    85       90       70
1      Bob    92       88       95
2  Charlie    78       82       80


In [39]:
df_unpivot = df_pivot.melt(id_vars = ["Student"], value_vars= ["Math", "Science", "History"], var_name = "Subject", value_name = "Score")

In [40]:
df_unpivot

,Student,Subject,Score
0,Alice,Math,85
1,Bob,Math,92
2,Charlie,Math,78
3,Alice,Science,90
4,Bob,Science,88
5,Charlie,Science,82
6,Alice,History,70
7,Bob,History,95
8,Charlie,History,80
